### Тематическое моделирование 



In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd

In [20]:
# neg_df = pd.read_csv('/content/drive/MyDrive/NLP/lessons4/negative.csv')
# pos_df = pd.read_csv('/content/drive/MyDrive/NLP/lessons4/positive.csv')

In [81]:
import csv
text = []
with open('/content/drive/MyDrive/NLP/lessons4/negative.csv', 'r') as f:
    data = csv.reader(x.replace('\0', '') for x in f)
    for row in data:
        text.append(''.join(row))

In [82]:
neg_df = pd.DataFrame(text, columns=['text'])

In [84]:
import csv
text = []
with open('/content/drive/MyDrive/NLP/lessons4/positive.csv', 'r') as f:
    data = csv.reader(x.replace('\0', '') for x in f)
    for row in data:
        text.append(''.join(row))

In [85]:
pos_df = pd.DataFrame(text, columns=['text'])

In [126]:
data = pos_df.append(neg_df, ignore_index = True, sort = False)

In [127]:
data.sample(5)

,text
201224,"415148745760579584;""1387814148"";""kellydead"";""н..."
143142,"409192805718700032;""1386394141"";""MiuShizumi"";""..."
254164,"422389081092276224;""1389540379"";""_Holographik_..."
84650,"Конечно на одних фруктах сижу)"";""1"";""0"";""0"";""0..."
129342,"411128184101273600;""1386855572"";""K_Shishlik"";""..."


In [88]:
len(data)

276051

In [128]:
topics = ['Россия', 'Мир', 'Экономика', 'Спорт', 'Культура', 'Путешествия', 
          'Наука и техника', 'Музыка']

In [129]:
import re
import numpy as np
from nltk.corpus import stopwords
from tqdm.notebook import tqdm
from multiprocessing import Pool
from pymystem3 import Mystem

In [130]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [92]:
! pip install pymorphy2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 55 kB 2.4 MB/s 
     |████████████████████████████████| 8.2 MB 9.7 MB/s 
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13723 sha256=0636771c4b6993a1c46c5265c29fc165a858019f33f3522a89f47a8f1c2be4a2
  Stored in directory: /root/.cache/pip/wheels/72/b0/3f/1d95f96ff986c7dfffe46ce2be4062f38ebd04b506c77c81b9
Successfully built docopt


In [93]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

In [131]:
words_regex = re.compile('\w+')

def find_words(text, regex = words_regex):
    tokens =  regex.findall(text.lower())
    return [w for w in tokens if w.isalpha() and len(w) >= 3]


stopwords_list = stopwords.words('russian')

def del_pattern(text, sub_):
    def del_user(text, sub_):
        return re.sub(sub_, '', text)
    vectorize_del_user = np.vectorize(del_user)
    return vectorize_del_user(text, sub_)

def lemmatize(words, lemmer = morph, stopwords = stopwords_list):
    lemmas = [lemmer.parse(w)[0].normal_form for w in words]
    return [w for w in lemmas if not w in stopwords 
            and w.isalpha()]

def preprocess(text):
    return (lemmatize(find_words(text)))

<input>:1: DeprecationWarning: invalid escape sequence \w
<input>:1: DeprecationWarning: invalid escape sequence \w
<input>:1: DeprecationWarning: invalid escape sequence \w
<input>:1: DeprecationWarning: invalid escape sequence \w
<input>:1: DeprecationWarning: invalid escape sequence \w
<input>:1: DeprecationWarning: invalid escape sequence \w
<input>:1: DeprecationWarning: invalid escape sequence \w
<input>:1: DeprecationWarning: invalid escape sequence \w
<input>:1: DeprecationWarning: invalid escape sequence \w
<input>:1: DeprecationWarning: invalid escape sequence \w
<input>:1: DeprecationWarning: invalid escape sequence \w
<input>:1: DeprecationWarning: invalid escape sequence \w
<input>:1: DeprecationWarning: invalid escape sequence \w
<input>:1: DeprecationWarning: invalid escape sequence \w
<input>:1: DeprecationWarning: invalid escape sequence \w
<input>:1: DeprecationWarning: invalid escape sequence \w
<ipython-input-131-d08dd02f65d4>:1: DeprecationWarning: invalid escape s

In [132]:
data.text.iloc[1]

'408906692693221377;"1386325927";"alinakirpicheva";"Да все-таки он немного похож на него. Но мой мальчик все равно лучше:D";"1";"0";"0";"0";"11825";"59";"31";"2"'

In [133]:
data['text'] = del_pattern(data['text'], "[a|b|c|d|e|f|g|h|i|j|k|l|m|n|o|p|q|r|s|t|u|v|w|x|y|z]")

In [134]:
print(preprocess(data.text.iloc[1]))

['всё', 'таки', 'немного', 'похожий', 'мальчик', 'всё', 'равно', 'хороший']


Ниже мы оставим только случайные 10,000 текстов из нашей коллекции, чтобы ускорить вычисления:

In [135]:
data = data.sample(10000)
data

,text
227940,";( почему я?!"";""-1"";""0"";""0"";""0"";""546"";""184"";""6..."
121667,"411060032772788224;""1386839323"";""AL"";""Девочки ..."
96691,и вот наконец с подругой которая видела их раз...
91702,"410639098899337216;""1386738965"";"""";""G — очень ..."
148190,# 逃走 ((
...,...
41896,"409763428899618817;""1386530189"";""16131"";""Списо..."
208693,"415885072269447168;""1387989702"";""_"";""@ а у мну..."
130557,#RT
209057,"415900283642736640;""1387993329"";""SK"";""я уже за..."


In [136]:
preprocessed_text = list(tqdm(map(preprocess, data['text']), total=len(data)))

  0%|          | 0/10000 [00:00<?, ?it/s]

In [137]:
data['text'] = preprocessed_text
data.sample(3)

,text
40173,"[апрель, измазать, весь, спина, белый, верить,..."
113501,"[дирекшионерфолловьдирекшионер, это, любовь, н..."
274492,"[шапочка, мочь, выбрать]"


## Модель LDA


Первая модель, которую мы рассмотрим, LDA - латентное размещение Дирихле. Воспользуемся реализацией из библиотеки gensim.

In [101]:
!pip install -U gensim

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 24.1 MB 1.5 MB/s 
  Attempting uninstall: gensim
    Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0


In [102]:
from gensim.models import *
from gensim import corpora

Модель использует векторное представление документов, например, мешок слов (bow), поэтому сперва создадим словарь:

In [138]:
dictionary = corpora.Dictionary(data['text'])

dictionary.filter_extremes(no_below = 10, no_above = 0.9, keep_n=None) # игнорируем слова, которые встречаются реже 10 раз или составляют более 0.9 словаря 
dictionary.save('lenta.dict')

Векторизуем документы:

In [139]:
corpus = [dictionary.doc2bow(text) for text in data['text']]
corpora.MmCorpus.serialize('lenta.model', corpus) 

In [156]:
print(corpus[:3])

[[(0, 1)], [(1, 1), (2, 1), (3, 1), (4, 1)], [(5, 1), (6, 1), (7, 1), (8, 1)]]


Теперь можем обучать модель:

In [140]:
%time lda = ldamodel.LdaModel(corpus, id2word=dictionary, num_topics=20, chunksize=50, update_every=1, passes=2)

CPU times: user 3 s, sys: 14.1 ms, total: 3.02 s
Wall time: 3.02 s


Посмотрим на получившиеся темы:

In [141]:
topics

['Россия',
 'Мир',
 'Экономика',
 'Спорт',
 'Культура',
 'Путешествия',
 'Наука и техника',
 'Музыка']

In [142]:
lda.show_topics(num_topics=10, num_words=10, formatted=True)

[(0,
  '0.431*"день" + 0.127*"сидеть" + 0.118*"думать" + 0.102*"блин" + 0.084*"час" + 0.075*"прийти" + 0.012*"тренировка" + 0.011*"палец" + 0.000*"рождение" + 0.000*"мочь"'),
 (9,
  '0.602*"хотеть" + 0.215*"вообще" + 0.124*"пойти" + 0.000*"новый" + 0.000*"сегодня" + 0.000*"нужно" + 0.000*"жить" + 0.000*"страна" + 0.000*"чувак" + 0.000*"забыть"'),
 (13,
  '0.298*"первый" + 0.292*"почему" + 0.098*"девочка" + 0.084*"праздник" + 0.054*"понедельник" + 0.053*"экзамен" + 0.028*"сначала" + 0.000*"январь" + 0.000*"весь" + 0.000*"минута"'),
 (12,
  '0.566*"стать" + 0.243*"увидеть" + 0.000*"ходить" + 0.000*"смотреть" + 0.000*"зал" + 0.000*"настроение" + 0.000*"танцевать" + 0.000*"большой" + 0.000*"купить" + 0.000*"парень"'),
 (16,
  '0.001*"чай" + 0.001*"похоже" + 0.001*"весна" + 0.001*"уровень" + 0.001*"окно" + 0.001*"ловить" + 0.001*"рабочий" + 0.001*"рядом" + 0.001*"странно" + 0.001*"особенно"'),
 (14,
  '0.678*"это" + 0.133*"говорить" + 0.107*"школа" + 0.028*"картинка" + 0.022*"проходить" + 0

На полученные темы можно посмотреть, изобразив их на плоскости с помощью библиотеки __pyLDAvis__. Чтобы спроецировать темы на плоскость, внутри применяется PCA-разложение.

In [143]:
!pip install pyLDAvis

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [144]:
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis


%time vis_data = gensimvis.prepare(lda, corpus, dictionary)
pyLDAvis.display(vis_data)

/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


CPU times: user 1.1 s, sys: 142 ms, total: 1.24 s
Wall time: 5.51 s


Посмотрим на метрики качества получившейся модели: перплексию и среднюю когерентность:

In [145]:
print(lda.log_perplexity(corpus))


-24.20112211853336


In [146]:
print('Персплексия: ', np.exp(lda.log_perplexity(corpus)))

Персплексия:  3.0873524332983906e-11


In [152]:
from gensim.models import CoherenceModel
# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda, texts=data['text'], dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

IndexError: ignored

In [151]:
coherence_model_lda = CoherenceModel(model=lda, texts=data['text'].values, dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('Средняя когерентность: ', coherence_lda)

Теперь можно подобрать оптимальное количество тем, опираясь на значение метрик:

In [114]:
import matplotlib.pyplot as plt
%matplotlib inline

In [125]:
topics_list = [5, 10, 15, 20, 25]
coherences = []

for num in tqdm(topics_list):
    lda = ldamodel.LdaModel(corpus, id2word=dictionary, num_topics=num, chunksize=50, update_every=1, passes=2)
    coherences.append(CoherenceModel(model=lda, texts=data['text'], dictionary=dictionary, coherence='c_v').get_coherence())

plt.plot(topics_list, coherences)
plt.xlabel("Число тем")
plt.ylabel("Средняя когерентность")
plt.show()